Nesta etapa, vamos transformar o dataset limpo (`dados_limpos.csv`) em um formato tidy data, seguindo os princípios:
- Cada variável é uma coluna.
- Cada observação é uma linha.
- Cada tipo de unidade observacional forma uma tabela.

Ao final, exportaremos o resultado em formato Parquet, mais eficiente e padronizado.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

In [ ]:
df = pd.read_csv("dados_limpos.csv")
print("Dimensões iniciais:", df.shape)

## 1) Preparação dos Dados em Formato Tidy

Padronização dos tipos de dados: converte colunas categóricas em category, numéricas inteiras em Int64 e valores decimais em float com duas casas decimais.

In [ ]:
cols_categoricas = ["TP_SEXO", "SG_UF_PROVA", "Q001", "Q002", "Q006", "NO_MUNICIPIO_PROVA"]
for col in cols_categoricas:
    df[col] = df[col].astype("category")

cols_inteiras = [
    "NU_ANO", "TP_FAIXA_ETARIA", "TP_ESTADO_CIVIL", "TP_COR_RACA",
    "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ESCOLA", "IN_TREINEIRO",
    "CO_UF_PROVA", "TP_PRESENCA_CN", "TP_PRESENCA_CH", "TP_PRESENCA_LC",
    "TP_PRESENCA_MT", "TP_LINGUA", "TP_STATUS_REDACAO",
    "NU_NOTA_COMP1", "NU_NOTA_COMP2", "NU_NOTA_COMP3",
    "NU_NOTA_COMP4", "NU_NOTA_COMP5", "NU_NOTA_REDACAO",
    "Q022", "Q024", "Q025", "CO_MUNICIPIO_PROVA"
]
df[cols_inteiras] = df[cols_inteiras].astype("Int64")

cols_float = [
    "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT",
    "PCT_ACERTO_CN", "PCT_ACERTO_CH", "PCT_ACERTO_LC", "PCT_ACERTO_MT", "PIB_MUNICIPIO"
]
df[cols_float] = df[cols_float].astype(float).round(2)

Normalização das variáveis TX_ACERTOS_*, que são sequências de 0 e 1 indicando acertos e erros.
Aqui são criadas colunas numéricas com o total de acertos e o total de questões por área (CN, CH, LC e MT).

In [ ]:
for area in ["CN", "CH", "LC", "MT"]:
    col = f"TX_ACERTOS_{area}"
    if col in df.columns:
        df[f"ACERTOS_{area}"] = df[col].astype(str).apply(lambda x: x.count("1") if pd.notna(x) else None)
        df[f"TOTAL_{area}"] = df[col].astype(str).apply(lambda x: len(x) if pd.notna(x) else None)

Transformação do dataset de formato wide para long, reorganizando as notas (NU_NOTA_*) para que cada linha represente um aluno em uma área específica (AREA_CONHECIMENTO).

In [ ]:
id_vars = [
    "NU_ANO", "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL",
    "TP_COR_RACA", "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ESCOLA",
    "IN_TREINEIRO", "CO_UF_PROVA", "SG_UF_PROVA", "TP_LINGUA",
    "TP_STATUS_REDACAO", "NU_NOTA_REDACAO", "Q001", 
    "Q002", "Q006", "Q022", "Q024", "Q025", "CO_MUNICIPIO_PROVA", "NO_MUNICIPIO_PROVA", "PIB_MUNICIPIO"
]

df_long_notas = df.melt(
    id_vars=id_vars,
    value_vars=["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="NOTA"
)
df_long_notas["AREA_CONHECIMENTO"] = df_long_notas["AREA_CONHECIMENTO"].str.replace("NU_NOTA_", "")

Aplicação da mesma transformação (melt) para as variáveis de percentual de acerto (PCT_ACERTO_*), número de acertos (ACERTOS_*) e total de questões (TOTAL_*).

In [ ]:
df_long_pct = df.melt(
    id_vars=id_vars,
    value_vars=["PCT_ACERTO_CN", "PCT_ACERTO_CH", "PCT_ACERTO_LC", "PCT_ACERTO_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="PCT_ACERTO"
)
df_long_pct["AREA_CONHECIMENTO"] = df_long_pct["AREA_CONHECIMENTO"].str.replace("PCT_ACERTO_", "")

df_long_acertos = df.melt(
    id_vars=id_vars,
    value_vars=["ACERTOS_CN", "ACERTOS_CH", "ACERTOS_LC", "ACERTOS_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="ACERTOS"
)
df_long_acertos["AREA_CONHECIMENTO"] = df_long_acertos["AREA_CONHECIMENTO"].str.replace("ACERTOS_", "")

df_long_total = df.melt(
    id_vars=id_vars,
    value_vars=["TOTAL_CN", "TOTAL_CH", "TOTAL_LC", "TOTAL_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="TOTAL_QUESTOES"
)
df_long_total["AREA_CONHECIMENTO"] = df_long_total["AREA_CONHECIMENTO"].str.replace("TOTAL_", "")

Combinação das tabelas geradas na etapa anterior, unindo notas, acertos e percentuais em um único dataset tidy (df_tidy).

In [ ]:
df_tidy = df_long_notas.copy()
df_tidy["PCT_ACERTO"] = df_long_pct["PCT_ACERTO"]
df_tidy["ACERTOS"] = df_long_acertos["ACERTOS"]
df_tidy["TOTAL_QUESTOES"] = df_long_total["TOTAL_QUESTOES"]

print("Dimensões finais (tidy):", df_tidy.shape)
df_tidy.head()

Exportação do dataset final em formato Parquet, garantindo compactação, preservação dos tipos de dados e maior eficiência de leitura.


In [ ]:
df_tidy.to_parquet("dados_tidy.parquet", index=False)

In [ ]:
area_labels = {
    "CN": "Ciências da Natureza",
    "CH": "Ciências Humanas",
    "LC": "Linguagens e Códigos",
    "MT": "Matemática"
}

df_tidy["AREA_LABEL"] = df_tidy["AREA_CONHECIMENTO"].map(area_labels)

In [ ]:
df_tidy.info()

## 2) Consultas

In [ ]:
import duckdb

sns.set_theme(style="whitegrid", palette="colorblind", font_scale=1.15)

con = duckdb.connect(database=':memory:')
con.register('enem', df_tidy)

Análise de tendências temporais: evolução das notas médias por área ao longo dos anos.

In [ ]:
query_tendencia_label = """
SELECT 
  NU_ANO::INTEGER AS NU_ANO,
  AREA_LABEL,
  AVG(NOTA) AS NOTA
FROM enem
WHERE AREA_LABEL IS NOT NULL
  AND NOTA IS NOT NULL
GROUP BY NU_ANO, AREA_LABEL
ORDER BY AREA_LABEL, NU_ANO;
"""
media_ano_area = con.sql(query_tendencia_label).df()

media_ano_area = media_ano_area.sort_values(["AREA_LABEL", "NU_ANO"])

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=media_ano_area,
    x="NU_ANO",
    y="NOTA",
    hue="AREA_LABEL",
    marker="o",
    linewidth=2.5
)
plt.title("Evolução das notas médias por área (ENEM)", fontsize=14, weight="bold")
plt.xlabel("Ano do ENEM")
plt.ylabel("Nota média")
plt.legend(title="Área de Conhecimento", loc="best", frameon=True)
plt.tight_layout()
plt.show()


Comparação entre grupos: médias de nota por raça e área de conhecimento.

In [ ]:
cor_raca_labels = {
    0: "Não declarado",
    1: "Branca",
    2: "Preta",
    3: "Parda",
    4: "Amarela",
    5: "Indígena",
    6: "Não dispõe"
}

query_grupos_label = """
SELECT 
    TP_COR_RACA,
    AREA_LABEL,
    ROUND(AVG(NOTA), 2) AS media_nota
FROM enem
WHERE AREA_LABEL IS NOT NULL
  AND NOTA IS NOT NULL
GROUP BY TP_COR_RACA, AREA_LABEL
ORDER BY AREA_LABEL, media_nota DESC;
"""
grupos = con.sql(query_grupos_label).df()

grupos["TP_COR_RACA"] = grupos["TP_COR_RACA"].map(cor_raca_labels)

ordem_areas = grupos.groupby("AREA_LABEL")["media_nota"].mean().sort_values(ascending=False).index

plt.figure(figsize=(10, 6))
sns.barplot(
    data=grupos,
    x="AREA_LABEL",
    y="media_nota",
    hue="TP_COR_RACA",
    palette="colorblind",
    order=ordem_areas
)

plt.title("Comparação de Notas Médias por Raça e Área de Conhecimento", fontsize=14, weight="bold")
plt.xlabel("Área de Conhecimento")
plt.ylabel("Nota Média")
plt.legend(
    title="Cor/Raça",
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0,
    frameon=True
)
plt.tight_layout()
plt.show()

Análise de concentração: distribuição das notas por área.

In [ ]:
query_boxplot = """
SELECT 
    AREA_LABEL,
    NOTA
FROM enem
WHERE AREA_LABEL IS NOT NULL
  AND NOTA IS NOT NULL;
"""
boxplot_df = con.sql(query_boxplot).df()

ordem_areas = boxplot_df.groupby("AREA_LABEL")["NOTA"].mean().sort_values(ascending=False).index

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=boxplot_df,
    x="AREA_LABEL",
    y="NOTA",
    palette="colorblind",
    order=ordem_areas
)
plt.title("Distribuição das Notas por Área de Conhecimento", fontsize=14, weight="bold")
plt.xlabel("Área de Conhecimento")
plt.ylabel("Nota")
plt.tight_layout()
plt.show()

Ranking de desempenho médio por estado (SG_UF_PROVA).

In [ ]:
query_ranking = """
SELECT 
    SG_UF_PROVA,
    ROUND(AVG(NOTA), 2) AS media_nota
FROM enem
GROUP BY SG_UF_PROVA
ORDER BY media_nota DESC;
"""
ranking_estados = con.sql(query_ranking).df()
ranking_estados


Correlações e dependências entre variáveis.

In [ ]:
query_heatmap = """
SELECT 
    NOTA,
    PCT_ACERTO,
    PIB_MUNICIPIO,
    ACERTOS,
    TOTAL_QUESTOES
FROM enem
WHERE NOTA IS NOT NULL;
"""
heatmap_df = con.sql(query_heatmap).df()

corr = heatmap_df.corr().round(2)

sns.set_theme(style="white", font_scale=1.1)

plt.figure(figsize=(8, 6))
sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    center=0,
    linewidths=0.6,
    square=True,
    cbar_kws={"shrink": 0.8, "label": "Coeficiente de Correlação"}
)

plt.title("Matriz de Correlação entre Variáveis Numéricas", fontsize=14, weight="bold")
plt.tight_layout()
plt.show()


## 3) Análise Exploratório e Testes de Hipótese

### Análise Univariada
#### Pergunta de Pesquisa: Como é a distribuição das notas em cada área do conhecimento?

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df_tidy,
    x="AREA_LABEL",
    y="NOTA",
    palette="Set2",
    hue="AREA_LABEL",
    legend=False,
    width=0.6,
    showfliers=True
)

plt.title("Distribuição das notas por área de conhecimento", fontsize=14, weight="bold")
plt.xlabel("Área de Conhecimento")
plt.ylabel("Nota")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

stats_resumo = df_tidy.groupby("AREA_LABEL")["NOTA"].describe().round(2)
display(stats_resumo)

### Análise Bivariada

#### Pergunta de Pesquisa: O desempenho médio dos alunos cresce com o aumento da renda familiar (Q006)?

In [ ]:
map_renda_ord = {letra: i + 1 for i, letra in enumerate(list("ABCDEFGHIJKLMNOPQ"))}

df_tidy["RENDA_ORD"] = df_tidy["Q006"].map(map_renda_ord)

media_renda = (
    df_tidy.groupby(["AREA_LABEL", "RENDA_ORD"])["NOTA"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=media_renda,
    x="RENDA_ORD",
    y="NOTA",
    hue="AREA_LABEL",
    marker="o",
    palette="coolwarm"
)

plt.title("Relação entre renda familiar e notas médias por área", fontsize=14, weight="bold")
plt.xlabel("Faixa de Renda (1 = menor, 17 = maior)")
plt.ylabel("Nota média")
plt.legend(title="Área de Conhecimento", bbox_to_anchor=(1.02, 0.5), loc="center left")
plt.grid(alpha=0.4)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()

corr_renda = df_tidy[["RENDA_ORD", "NOTA"]].corr(method="spearman").iloc[0, 1]
print(f"Correlação de Spearman entre renda familiar e nota: {corr_renda:.2f}")

### Análise Multivariada

#### Pergunta de Pesquisa: As notas das quatro áreas (CN, CH, LC, MT) são fortemente correlacionadas?

In [ ]:
notas_pivot = df_tidy.pivot_table(
    index=["NU_ANO", "TP_SEXO", "CO_UF_PROVA"],
    columns="AREA_CONHECIMENTO",
    values="NOTA"
).reset_index()

corr_areas = notas_pivot[["CN", "CH", "LC", "MT"]].corr()

plt.figure(figsize=(6, 5))
sns.heatmap(
    corr_areas,
    annot=True,
    fmt=".2f",
    cmap="YlGnBu",
    cbar_kws={"shrink": 0.8}
)

plt.title("Correlação entre notas das áreas do ENEM", fontsize=13, weight="bold")
plt.tight_layout()
plt.show()

### Testes de Hipóteses

#### Escolaridade dos Pais
Estudantes com pais que possuem ensino superior apresentam notas de Matemática cerca de 15% maiores que os de pais com ensino fundamental.

In [ ]:
fundamental = ["A", "B", "C", "D"]
superior = ["F", "G"]

notas_mt = df_tidy[df_tidy["AREA_CONHECIMENTO"] == "MT"]

grupo_fundamental = notas_mt[notas_mt["Q001"].isin(fundamental) & notas_mt["Q002"].isin(fundamental)]["NOTA"]
grupo_superior = notas_mt[notas_mt["Q001"].isin(superior) & notas_mt["Q002"].isin(superior)]["NOTA"]


t_stat, p_val = ttest_ind(grupo_superior.dropna(), grupo_fundamental.dropna(), equal_var=False)
mean_diff = grupo_superior.mean() / grupo_fundamental.mean() - 1

print(f"T-statística: {t_stat:.2f}, p-valor: {p_val:.4f}")
print(f"Variação percentual média: {mean_diff*100:.1f}%")

if p_val < 0.05:
    print("Evidência significativa: rejeita H0, há diferença nas médias.")
else:
    print("Sem evidência significativa: não rejeita H0.")

#### Tipo de Escola
Alunos de escolas particulares têm desempenho em Linguagens aproximadamente 20% superior aos de escolas públicas.

In [ ]:
notas_lc = df_tidy[df_tidy["AREA_CONHECIMENTO"] == "LC"]

grupo_publica = notas_lc[notas_lc["TP_ESCOLA"] == 1]["NOTA"]
grupo_privada = notas_lc[notas_lc["TP_ESCOLA"] == 2]["NOTA"]

t_stat, p_val = ttest_ind(grupo_privada.dropna(), grupo_publica.dropna(), equal_var=False)
mean_diff = grupo_privada.mean() / grupo_publica.mean() - 1

print(f"T-statística: {t_stat:.2f}, p-valor: {p_val:.4f}")
print(f"Variação percentual média: {mean_diff*100:.1f}%")

if p_val < 0.05:
    print("Evidência significativa: rejeita H0, alunos de escolas privadas têm médias maiores.")
else:
    print("Sem evidência significativa: não rejeita H0.")

#### Renda Familiar
Há correlação positiva entre renda familiar e a nota de Matemática

In [ ]:
notas_mt = df_tidy[df_tidy["AREA_CONHECIMENTO"] == "MT"]

corr_renda_mt = notas_mt[["RENDA_ORD", "NOTA"]].corr(method="spearman").iloc[0, 1]
print(f"Correlação de Spearman entre renda e nota (MT): {corr_renda_mt:.2f}")

if corr_renda_mt > 0.5:
    print("Evidência de forte correlação positiva (r > 0.5).")
else:
    print("Correlação fraca ou moderada (r ≤ 0.5).")

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=notas_mt,
    x="RENDA_ORD",
    y="NOTA",
    palette="viridis",
    hue="RENDA_ORD",
    legend=False,
)
plt.title("Distribuição das notas de Matemática por faixa de renda", fontsize=13, weight="bold")
plt.xlabel("Faixa de renda (A–Q → 1–17)")
plt.ylabel("Nota em Matemática")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
